<a href="https://colab.research.google.com/github/andresdevml/dl_writing_quality/blob/main/data_scaler_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importamos librerias de interes**

In [ ]:

import keras

import tensorflow as tf

from keras.models import Sequential

from keras.layers import LSTM

from keras.layers import Dense

from keras.layers import RepeatVector

from keras.layers import TimeDistributed

import numpy as np

import pandas as pd

from sklearn import svm

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt

from scipy import interpolate

from google.colab import drive

import pickle

# **Importamos la data**

In [ ]:
# conectamos al drive
drive.mount('/content/drive')

# Definimos el directorio
dir_data_kaggle='./drive/MyDrive/lwpwq/data'

# extraemos la data

# open a file, where you stored the pickled data
file = open(dir_data_kaggle+'/train_data.pkl', 'rb')

# dump information to that file
train_data = pickle.load(file)

# close the file
file.close()

df_train_logs=pd.read_csv(filepath_or_buffer=dir_data_kaggle+'/train_logs.csv')

df_train_scores=pd.read_csv(filepath_or_buffer=dir_data_kaggle+'/train_scores.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# vamos a probar el codigo con 5 ids nada mas


id_list_test=list(train_data.keys())[:5]


train_data={key:train_data[key] for key in id_list_test}

# **Escalamos la data**

In [ ]:
# sabemos que dyn data tiene 8 columnas

# ajustamos y guardamos los scalers

list_scaler=[]

for k in range(8):

  list_seq=[]
  for id in train_data:
    seq=list(train_data[id]['dyn_data'][:,k])
    list_seq=list_seq+seq

  list_scaler.append(
                      MinMaxScaler(feature_range=(0, 5))
                      .fit(
                               np.array(list_seq).reshape(-1,1)
                                                                  )
                                                                               )

In [ ]:
# aplicamos el escaler para cada secuencia en cada columnsa de dyn data

for id in train_data:

  dyn_data=train_data[id]['dyn_data']

  train_data[id]['dyn_data']=np.hstack(tup=tuple(

                  [ list_scaler[k].transform(dyn_data[:,k].reshape(-1,1)) for k in range(8) ]

                                              )
                                                  )

#**Aplicamos Padding sobre la data**

In [ ]:
# determianmos la longitud maxima de las secuencias

max_len=int(df_train_logs['event_id'].max()*1.20)

print(max_len)


15451


In [ ]:
# generamos la lista para cada columna

list_data=[]

for column in range(8):

  list_seq=[]

  for id in train_data:
    list_seq.append(train_data[id]['dyn_data'][:,column])

  list_data.append(list_seq)

In [ ]:
# aplicamos padding a cada columna y juntamos las columns
pad_list_data=[tf.keras.utils.pad_sequences(
                                            sequences=column,
                                              maxlen=max_len,
                                              dtype='float64',
                                              padding='post',
                                              truncating='pre',
                                              value=-1

                                                    ) for column in list_data
                                                              ]


pad_train_data=np.hstack(tup=tuple(pad_list_data)).reshape(len(train_data),-1,8)

# **Exportamos la data**